In [ ]:
%pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 44.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
from pgmpy.models import BayesianNetwork, DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [ ]:
model = DiscreteBayesianNetwork([("Car", "Host"), ("Choice", "Host")])

In [ ]:
cpd_car = TabularCPD(variable = "Car", variable_card = 3,
                     values = [[1/3], [1/3], [1/3]],
                     state_names = {"Car": [1, 2, 3]})

In [ ]:
cpd_choice = TabularCPD(variable = "Choice", variable_card = 3,
                        values = [[1/3], [1/3], [1/3]],
                        state_names = {"Choice": [1, 2, 3]})

In [ ]:
host_values = np.zeros((3, 9))

In [ ]:
host_doors = [1, 2, 3]
for i, car in enumerate(host_doors):
  for j, choice in enumerate(host_doors):
    valid = [door for door in host_doors if door != car and door != choice]
    for v in valid:
      host_values[v - 1][3 * i + j] = 1 / len(valid)

cpd_host = TabularCPD(variable = "Host", variable_card = 3,
                      values = host_values,
                      evidence = ["Car", "Choice"],
                      evidence_card = [3, 3],
                      state_names = {"Host": [1, 2, 3], "Car": [1, 2, 3], "Choice": [1, 2, 3]})

In [ ]:
model.add_cpds(cpd_car, cpd_choice, cpd_host)

In [ ]:
model.check_model()

True

In [ ]:
inference = VariableElimination(model)

In [ ]:
query_result = inference.query(variables = ["Car"], evidence = {"Choice": 2, "Host": 1})

In [ ]:
print(query_result)

+--------+------------+
| Car    |   phi(Car) |
+========+============+
| Car(1) |     0.0000 |
+--------+------------+
| Car(2) |     0.3333 |
+--------+------------+
| Car(3) |     0.6667 |
+--------+------------+
